5/15/19

Testing the extraction of CWL data to (1) construct a way to programatically identify the top 3 placing teams. This will be done by utilizing the end time of matches. Special handling will have to be incorporated for the Grand Final matches as they can extend to 2 best of 5 series if the team coming out of the loseres bracket wins the first best of 5.

Notes accompanying this processing is page one of the notebook.  

In [1]:
import pandas as pd
import os

In [3]:
pl1 = pd.read_csv('../data_imports/proleague1.csv')

In [5]:
pl1.columns

Index(['Unnamed: 0', 'match id', 'series id', 'end time', 'duration (s)',
       'mode', 'map', 'team', 'player', 'win?', 'score', 'kills', 'deaths',
       '+/-', 'k/d', 'kills per 10min', 'deaths per 10min', 'player score',
       'player spm', 'ekia', 'assists', 'headshots', 'suicides', 'team kills',
       'team deaths', 'kills (stayed alive)', 'hits', 'shots', 'accuracy (%)',
       'num lives', 'time alive (s)', 'avg time per life (s)', 'fave weapon',
       'fave specialist', 'fave scorestreaks', 'hill time (s)',
       'hill captures', 'hill defends', 'snd rounds', 'snd firstbloods',
       'snd firstdeaths', 'snd survives', 'bomb pickups', 'bomb plants',
       'bomb defuses', 'bomb sneak defuses', 'snd 1-kill round',
       'snd 2-kill round', 'snd 3-kill round', 'snd 4-kill round',
       'ctrl rounds', 'ctrl firstbloods', 'ctrl firstdeaths', 'ctrl captures',
       '2-piece', '3-piece', '4-piece', '4-streak', '5-streak', '6-streak',
       '7-streak', '8+-streak'],
      dt

In [6]:
cols = ['series id', 'match id', 'end time', 'team', 'win?']

In [10]:
df = pl1[cols].copy()

In [11]:
df['win?'] = df['win?'].map({'W': 1, 'L':0})

In [17]:
df['end time'] = pd.to_datetime(df['end time'])

In [18]:
df.dtypes

series id                 object
match id                  object
end time     datetime64[ns, UTC]
team                      object
win?                       int64
dtype: object

In [22]:
df['end time'].nlargest(11)

2250   2019-03-01 01:00:59+00:00
2251   2019-03-01 01:00:59+00:00
2252   2019-03-01 01:00:59+00:00
2253   2019-03-01 01:00:59+00:00
2254   2019-03-01 01:00:59+00:00
2255   2019-03-01 01:00:59+00:00
2256   2019-03-01 01:00:59+00:00
2257   2019-03-01 01:00:59+00:00
2258   2019-03-01 01:00:59+00:00
2259   2019-03-01 01:00:59+00:00
2240   2019-03-01 00:46:48+00:00
Name: end time, dtype: datetime64[ns, UTC]

In [28]:
# can be useful for index slicing desired nlargest().
lst10 = df['end time'].nlargest(10).index

In [45]:
# slices the last match played by utilizing the index obtained with the nlargest() method above.
df.loc[lst10]

,series id,match id,end time,team,win?
2250,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,eUnited,1
2251,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,eUnited,1
2252,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,eUnited,1
2253,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,eUnited,1
2254,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,eUnited,1
2255,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,Splyce,0
2256,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,Splyce,0
2257,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,Splyce,0
2258,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,Splyce,0
2259,pro-w4-14,18047717744453498552,2019-03-01 01:00:59+00:00,Splyce,0


In [46]:
# slices the rows with matches played on or after a certain day. Can be useful for only utilizing data
#  on championship sunday. Should probably be step in in cleaning the data to reduce size.
df.loc[df['end time'] >= '2019-03-01'].head()

,series id,match id,end time,team,win?
2220,pro-w4-14,13529199086143555601,2019-03-01 00:01:27+00:00,eUnited,1
2221,pro-w4-14,13529199086143555601,2019-03-01 00:01:27+00:00,eUnited,1
2222,pro-w4-14,13529199086143555601,2019-03-01 00:01:27+00:00,eUnited,1
2223,pro-w4-14,13529199086143555601,2019-03-01 00:01:27+00:00,eUnited,1
2224,pro-w4-14,13529199086143555601,2019-03-01 00:01:27+00:00,eUnited,1


In [58]:
test = df.groupby(['series id', 'match id', 'end time', 'team'])['win?'].sum()/5

In [84]:
df = test.reset_index()

df.groupby(['series id', 'team'])['win?'].sum().reset_index()

# need to figure out how to get the end time of the last map for each series and create a column for series
# end time.

last_map_time = df.groupby('match id')['end time'].max()

In [104]:
t2 = df.groupby(['series id', 'team'])['win?'].sum().reset_index()

In [108]:
t2.filter(t2['win?'].sum() >= 3)

TypeError: 'numpy.bool_' object is not iterable

In [134]:
t2['series win'].map(lambda x: 1 if x['win?'] >= 3 else 0, t2['win?'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [130]:
t2

,series id,team,win?,series win
0,pro-w1-1,Reciprocity,0.0,<filter object at 0x000001F69212C710>
1,pro-w1-1,Red Reserve,3.0,<filter object at 0x000001F69212C710>
2,pro-w1-10,Evil Geniuses,0.0,<filter object at 0x000001F69212C710>
3,pro-w1-10,OpTic Gaming,3.0,<filter object at 0x000001F69212C710>
4,pro-w1-11,Midnight,3.0,<filter object at 0x000001F69212C710>
5,pro-w1-11,UYU,0.0,<filter object at 0x000001F69212C710>
6,pro-w1-12,OpTic Gaming,3.0,<filter object at 0x000001F69212C710>
7,pro-w1-12,Reciprocity,1.0,<filter object at 0x000001F69212C710>
8,pro-w1-13,Luminosity,1.0,<filter object at 0x000001F69212C710>
9,pro-w1-13,Red Reserve,3.0,<filter object at 0x000001F69212C710>
